## CS310 Natural Language Processing
## Lab 9: Explore BERT

In [12]:
from pprint import pprint
from typing import List
import torch
import torch.nn.functional as F

### T1. Explore Pretrained BERT Model

In this task, you will explore the pretrained BERT model using the Hugging Face Transformers library. 

First, you will load a pretrained BERT model and the correponding tokenizer. If you use the default model string `'bert-base-uncased'`, it will automatically download the model.

In our case, to avoid any network issue, you can follow these steps to load the model locally:
- Download the `bert-base-uncased.zip` file from the course website and unzip it to the folder `bert-base-uncased` in the same directory as this notebook. 
- When you load the model, you simply specify the folder path `bert-base-uncased/` (which contains all model files) to the `from_pretrained()` function. 
- *Note* that don't exclude the last `/` in the path.

In [43]:
from transformers import BertTokenizer, BertModel

bert_model = BertModel.from_pretrained('bert-base-uncased/',output_hidden_states=True) # Make sure you download the model files first
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased/')

Let's start by counting the number of parameters in the model.

In [3]:
n_tensors = 0
for param in bert_model.parameters():
    n_tensors += 1

print("Number of tensors: ", n_tensors)

Number of tensors:  199


In [4]:
n_params = 0
for param in bert_model.parameters():
    n_params += param.numel()

print("Number of parameters: ", n_params)

Number of parameters:  109482240


Next, if you are interested in how the parameters are organized, you can print the model's `_modules` attribute.

In [5]:
print(bert_model._modules)

OrderedDict([('embeddings', BertEmbeddings(
  (word_embeddings): Embedding(30522, 768, padding_idx=0)
  (position_embeddings): Embedding(512, 768)
  (token_type_embeddings): Embedding(2, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)), ('encoder', BertEncoder(
  (layer): ModuleList(
    (0-11): 12 x BertLayer(
      (attention): BertAttention(
        (self): BertSelfAttention(
          (query): Linear(in_features=768, out_features=768, bias=True)
          (key): Linear(in_features=768, out_features=768, bias=True)
          (value): Linear(in_features=768, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (output): BertSelfOutput(
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (intermediate)

In theory, you can access the parameters at any layer of the model, by specifying the layer name and index. 

For example, if you want to check the the query matrix $W^Q$ in the self-attention layer of the first transformer block, you can do the following:

In [44]:
pprint(bert_model._modules['encoder']._modules['layer'][0]._modules['attention']._modules['self']._modules['query'])

Linear(in_features=768, out_features=768, bias=True)


As you can see, the $W^Q$ matrix is implemented as a `nn.Linear` module.

Also, the same inquiry can be simplified by using the `get_submodule()` function.

In [10]:
W_q = bert_model.get_submodule('encoder.layer.0.attention.self.query')
print(W_q)
print(W_q.weight.shape)

Linear(in_features=768, out_features=768, bias=True)
torch.Size([768, 768])


### T2. Get Contextual Embeddings from BERT

Let's move on and use the BERT model to get contextual embeddings for given texts.

First, we prepare some sentences:

In [13]:
text = (
        'I have a new CPU!\n'
        'I have a new Intel CPU!\n'
        'I have a new GPU!\n'
        'I have a new NVIDIA GPU!'
    )

sentences = text.split('\n')
pprint(sentences)

['I have a new CPU!',
 'I have a new Intel CPU!',
 'I have a new GPU!',
 'I have a new NVIDIA GPU!']


Try the `tokenize()` function of the previously initialized BERT tokenizer on each sentence:

In [15]:
### START YOUR CODE ###
tokens_in_string: List[str] = [bert_tokenizer.tokenize(sentence) for sentence in sentences]
### END YOUR CODE ###

# Test
pprint(tokens_in_string)
# You should expect to see the following output:
# [['i', 'have', 'a', 'new', 'cpu', '!'],
#  ['i', 'have', 'a', 'new', 'intel', 'cpu', '!'],
#  ['i', 'have', 'a', 'new', 'gp', '##u', '!'],
#  ['i', 'have', 'a', 'new', 'n', '##vid', '##ia', 'gp', '##u', '!']]

[['i', 'have', 'a', 'new', 'cpu', '!'],
 ['i', 'have', 'a', 'new', 'intel', 'cpu', '!'],
 ['i', 'have', 'a', 'new', 'gp', '##u', '!'],
 ['i', 'have', 'a', 'new', 'n', '##vid', '##ia', 'gp', '##u', '!']]


**Note** that "CPU" and "Intel" are recognized as whole words, but "NVIDIA" and "GPU" are not. Thus, they appear as subwords such as "##u" "##vid" in the results

The above results are not integer token IDs yet, so now use the `batch_encode()` function, with argument `return_tensors='pt'`, to convert each sentence to integer token IDs. Here `'pt'` is for PyTorch tensors.

**Note**:
- Each token is represented as an integer in `torch.int64` data type.
- By default, the tokenizer adds special tokens `[CLS]` and `[SEP]` to the beginning and end of each sentence, which correpond to the token ID `101` and `102`, respectively.

In [16]:
### START YOUR CODE ###
token_ids_list: List[torch.Tensor] = [
    bert_tokenizer.batch_encode_plus([sentence], return_tensors='pt')['input_ids'] 
    for sentence in sentences
]
### END YOUR CODE ###

# Test
print(token_ids_list[0].dtype)
pprint(token_ids_list)

# You should expect to see the following output:
# torch.int64
# [tensor([[  101,  1045,  2031,  1037,  2047, 17368,   999,   102]]),
#  tensor([[  101,  1045,  2031,  1037,  2047, 13420, 17368,   999,   102]]),
#  tensor([[  101,  1045,  2031,  1037,  2047, 14246,  2226,   999,   102]]),
#  tensor([[  101,  1045,  2031,  1037,  2047,  1050, 17258,  2401, 14246,  2226,
#            999,   102]])]

torch.int64
[tensor([[  101,  1045,  2031,  1037,  2047, 17368,   999,   102]]),
 tensor([[  101,  1045,  2031,  1037,  2047, 13420, 17368,   999,   102]]),
 tensor([[  101,  1045,  2031,  1037,  2047, 14246,  2226,   999,   102]]),
 tensor([[  101,  1045,  2031,  1037,  2047,  1050, 17258,  2401, 14246,  2226,
           999,   102]])]


So now `"CPU"` is tokenized to `17368`, `"Intel"` to `13420`, while `"GPU"` to `[14246, 2226]`, and `"NVIDIA"` to `[1050, 17258,  2401]`.

You can use the `ids_to_tokens` dictionary to map integer token IDs back to token strings, and use `decode()` function to convert a list of token IDs back to a sentence.

In [17]:
print(bert_tokenizer.ids_to_tokens[101])
print(bert_tokenizer.ids_to_tokens[102])
print(bert_tokenizer.ids_to_tokens[17368])
print(bert_tokenizer.decode(token_ids_list[0].squeeze().tolist()))

[CLS]
[SEP]
cpu
[CLS] i have a new cpu! [SEP]


Note that in last example above, we `squeeze` the token IDs first, becaseu the encoded IDs are of dimension $1\times N$, where $N$ is sentence length, because PyTorch uses first dimension as batch size.

It indicates that we can tokenize multiple sentences in one batch by using the `batch_encode_plus()` function, and specify the argument `padding=True` to pad all sentences to the same length.

In [18]:
encoded_sentences = bert_tokenizer.batch_encode_plus(sentences, return_tensors='pt', padding=True, return_attention_mask=False, return_token_type_ids=False)
print(encoded_sentences)

{'input_ids': tensor([[  101,  1045,  2031,  1037,  2047, 17368,   999,   102,     0,     0,
             0,     0],
        [  101,  1045,  2031,  1037,  2047, 13420, 17368,   999,   102,     0,
             0,     0],
        [  101,  1045,  2031,  1037,  2047, 14246,  2226,   999,   102,     0,
             0,     0],
        [  101,  1045,  2031,  1037,  2047,  1050, 17258,  2401, 14246,  2226,
           999,   102]])}


As you can see, the returned dictionary contains an item keyed by `'input_ids'`, which is exactly the token IDs we need. 

**Note**:
- It is a tensor of shape $B\times N$, where $B$ is the batch size (here, $B=4$) and $N$ is the maximum sentence length in the batch.
- The default padding token is `0`.


In the above example, we deliberately set `return_attention_mask=False` to show simpler results. 

If you set it to `True`, then then returned dictionary will also contain an item keyed by `'attention_mask'`, which is a tensor of shape $B\times N$ with `1` for real tokens and `0` for padding tokens. This information is useful for follow-up computations.

Try if you can get the attention mask tensor:

In [19]:
### START YOUR CODE ###
encoded_sentences = bert_tokenizer.batch_encode_plus(
    sentences, 
    return_tensors='pt',  # 'pt' specifies PyTorch tensors
    padding=True,  # This ensures padding to the longest sentence
    truncation=True,  # Truncates longer sequences
    return_attention_mask=True  # Generate attention masks
)

# Extract the attention mask
attn_mask = encoded_sentences['attention_mask']
### END YOUR CODE ###

# Test
print(attn_mask)
# You should expect to see the following output:
# tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
#         [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
#         [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
#         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


Now, let's obtain the contextual embeddings for the four target words `"CPU"`, `"Intel"`, `"NVIDIA"`, and `"GPU"` in our sentences.

First, pass the token IDs in one batch to the BERT model to get the output object, which has a `last_hidden_state` attribute that contains the contextual embeddings.

**Note**:
- You can manually specify `input_ids` and `attention_mask` as the input arguments to the model.
- Or you can directly pass the dictionary returned by `batch_encode_plus()` to the model, and use the `**` operator as most tutorials did:
  - `outputs = model(**encoded_sentences)`

In [20]:
bert_model.eval()

with torch.no_grad():
    ### START YOUR CODE ###
    outputs =bert_model(**encoded_sentences) 
    ### END YOUR CODE ###


# Test
print(outputs.last_hidden_state.shape)
# You should expect to see the following output:
# torch.Size([4, 12, 768])

torch.Size([4, 12, 768])


Next, for ``"CPU"`` and ``"Intel"``, you can directly use the output vectors at the corresponding positions, because they are recognized as whole words.

Compute the average vector of `"CPU"`s in the first two sentences, and compute its cosine similarity with the vector of `"Intel"`.

*Hint*:
- Use `F.cosine_similarity()` function

In [25]:
### START YOUR CODE ###



vec_cpu1 = outputs.last_hidden_state[0][5] 
vec_cpu2 = outputs.last_hidden_state[1][6] 
vec_cpu_avg = (vec_cpu1 + vec_cpu2) / 2


vec_intel = outputs.last_hidden_state[1][5]  


cos_cpu_intel = F.cosine_similarity(vec_cpu_avg.unsqueeze(0), vec_intel.unsqueeze(0))

### END YOUR CODE ###

# Test
print('cos_cpu_intel:', cos_cpu_intel.item())
# You should expect to see the following output:
# cos_cpu_intel: 0.7551645636558533

cos_cpu_intel: 0.7551645636558533


For `"NVIDIA"` and `"GPU"`, it's a bit trickier, as you need to use the sum of subword vectors to get the vector of the whole word.

In sentence 3, `"GPU"` is tokenized to `[14246, 2226]`, so you need to sum the vectors at these two positions.

In sentence 4, `"NVIDIA"` is tokenized to `[1050, 17258,  2401]`, so you need to sum the vectors at these three positions.

In [39]:
### START YOUR CODE ###
vec_gpu1 = outputs.last_hidden_state[2][5]  
vec_gpu2 = outputs.last_hidden_state[2][6]  
vec_gpu3 = outputs.last_hidden_state[3][8]  
vec_gpu4 = outputs.last_hidden_state[3][9] 
vec_gpu = (vec_gpu1 + vec_gpu2  +vec_gpu3+vec_gpu4)/4

vec_nv1 = outputs.last_hidden_state[3][5]  
vec_nv2 = outputs.last_hidden_state[3][6]  
vec_nv3 = outputs.last_hidden_state[3][7] 
vec_nvidia = (vec_nv1 + vec_nv2 + vec_nv3 )/3 

cos_gpu_nv = F.cosine_similarity(vec_gpu.unsqueeze(0), vec_nvidia.unsqueeze(0))

### END YOUR CODE ###

# Test
print('cos_gpu_nv:', cos_gpu_nv.item())
# You should expect to see the following output:
# cos_gpu_nv: 0.7273837327957153


cos_gpu_nv: 0.7273837327957153


Now let's see if `"NVIDIA"` is closer to `"GPU"` than `"CPU"`, and vice versa for `"Intel"`.

In [40]:
### START YOUR CODE ###
cos_cpu_nv = F.cosine_similarity(vec_cpu_avg.unsqueeze(0), vec_nvidia.unsqueeze(0))
cos_gpu_intel = F.cosine_similarity(vec_gpu.unsqueeze(0), vec_intel.unsqueeze(0))
### END YOUR CODE ###

# Test
print('cos_cpu_nv:', cos_cpu_nv.item())
print('cos_gpu_intel:', cos_gpu_intel.item())
# You should expect to see the following output:
# cos_cpu_nv: 0.5931224226951599
# cos_gpu_intel: 0.5778647661209106

cos_cpu_nv: 0.5931224822998047
cos_gpu_intel: 0.5778650045394897


That's interesting, right?

How about the distance between the two products `"CPU"` and `"GPU"`? or between the two companies `"Intel"` and `"NVIDIA"`? Check it out yourself.

In [45]:
### START YOUR CODE ###
cos_cpu_gpu = F.cosine_similarity(vec_cpu_avg.unsqueeze(0), vec_gpu.unsqueeze(0))
cos_intel_nv = F.cosine_similarity(vec_intel.unsqueeze(0), vec_nvidia.unsqueeze(0))

### END YOUR CODE ###

# Test
print('cos_cpu_gpu:', cos_cpu_gpu.item())
print('cos_intel_nv:', cos_intel_nv.item())
# You should expect to see the following output:
# cos_cpu_gpu: 0.6914964914321899
# cos_intel_nv: 0.6179742813110352

cos_cpu_gpu: 0.6914966106414795
cos_intel_nv: 0.617974579334259


### T3. Access all hidden states

Let's be more adventurous and access all hidden states returned by the BERT model.

*Hint*: Simply set the argument `output_hidden_states=True` when calling the model.

In [46]:
bert_model.eval()

with torch.no_grad():
    ### START YOUR CODE ###
    outputs = bert_model(**encoded_sentences)
    ### END YOUR CODE ###

# Test
print(type(outputs.hidden_states))
print(len(outputs.hidden_states))
print(outputs.hidden_states[-1].shape)
print(outputs.hidden_states[-2].shape)
print(outputs.hidden_states[-3].shape)
print(outputs.hidden_states[-4].shape)

# You should expect to see the following output:
# <class 'tuple'>
# 13
# torch.Size([4, 12, 768])
# torch.Size([4, 12, 768])
# torch.Size([4, 12, 768])
# torch.Size([4, 12, 768])

<class 'tuple'>
13
torch.Size([4, 12, 768])
torch.Size([4, 12, 768])
torch.Size([4, 12, 768])
torch.Size([4, 12, 768])


Compute the average vector of the word `"CPU"` in the first sentence, using the hidden states of the last **four** layer.

In [47]:
### START YOUR CODE ###
cpu_pos_1 = 5
hidden_state_layer_13 = outputs.hidden_states[-1] 
hidden_state_layer_12 = outputs.hidden_states[-2]  
hidden_state_layer_11 = outputs.hidden_states[-3]  
hidden_state_layer_10 = outputs.hidden_states[-4]  

vec_cpu_13 = hidden_state_layer_13[0][cpu_pos_1]  
vec_cpu_12 = hidden_state_layer_12[0][cpu_pos_1] 
vec_cpu_11 = hidden_state_layer_11[0][cpu_pos_1] 
vec_cpu_10 = hidden_state_layer_10[0][cpu_pos_1]  

vec_cpu_avg_last4 = (vec_cpu_10 + vec_cpu_11 + vec_cpu_12 + vec_cpu_13) / 4

cos = F.cosine_similarity(vec_cpu_avg_last4, vec_cpu_avg, dim=0) 
### END YOUR CODE ###

# Test
cos = F.cosine_similarity(vec_cpu_avg_last4, vec_cpu_avg, dim=0)
print('cos:', cos.item())

# You should expect to see the following output:
# cos: 0.9002149701118469

cos: 0.9002149701118469
